In [1]:
from fastembed import TextEmbedding
from qdrant_client import QdrantClient, models


/Users/chenyuan/anaconda3/envs/llm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

client = QdrantClient("http://localhost:6333") #connecting to local Qdrant instance


In [3]:
model_handle = "jinaai/jina-embeddings-v2-small-en"
EMBEDDING_DIMENSIONALITY = 512


# Q1. Embedding the query


In [4]:
# 輸入查詢句子
query = "I just discovered the course. Can I join now?"
embed_model = TextEmbedding(model_name="jinaai/jina-embeddings-v2-small-en")

# 計算嵌入向量
embedding = list(embed_model.embed([query]))[0]  # 取出第一個結果

min_value = min(embedding)
print(min_value)


-0.11726374368207196


# Q2. Cosine similarity with another vector


In [5]:

import numpy as np

# 文字查詢與文件
doc = "Can I still join the course after the start date?"

# 各自嵌入
q_vector = list(embed_model.embed([query]))[0]
doc_vector = list(embed_model.embed([doc]))[0]

# 計算 cosine similarity（等同 dot product，因為已正規化）
similarity = np.dot(q_vector, doc_vector)

print(similarity)

0.9008529058287051


# Q3. Ranking by cosine


In [6]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}
 ]

In [7]:
# 從 documents 中擷取所有問題
questions = [doc['question'] for doc in documents]

collection_name = "question3"

# 2. 建立 Qdrant collection
client.delete_collection(collection_name=collection_name)

client.create_collection(  # recreate 可避免重複建立
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)



# 3. 上傳向量與 payload
points = [
    models.PointStruct(id=idx,
                    vector=models.Document(text=doc['text'], model=model_handle),
                    payload={
                        "question": doc['question'],
                        "text": doc['text'],
                        "course": doc['course']
                        }
                    )
    for idx, doc in enumerate(documents)
]

client.upsert(collection_name=collection_name, points=points)

def search(query, limit=1):
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        limit=limit,
        with_payload=True
    )
    return results

# 4. 查詢最相似的問句

res = search("I just discovered the course. Can I join now?", limit=1)
# print("最相似的文件 id:", res[0].id)
# print("內容:", res[0].payload)

print(res)
point = res.points[0]

print(f"最相似文件 ID: {point.id}")
print(f"相似度分數: {point.score:.4f}")
print(f"問題: {point.payload['question']}")
print(f"內容摘要: {point.payload['text'][:200]}...")


points=[ScoredPoint(id=1, version=0, score=0.81823784, payload={'question': 'Course - Can I follow the course after it finishes?', 'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.', 'course': 'data-engineering-zoomcamp'}, vector=None, shard_key=None, order_value=None)]
最相似文件 ID: 1
相似度分數: 0.8182
問題: Course - Can I follow the course after it finishes?
內容摘要: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing fo...


# Q4. Ranking by cosine, version two


In [8]:
# 從 documents 中擷取所有問題
questions = [doc['question'] for doc in documents]

collection_name = "question4"

# 2. 建立 Qdrant collection
client.delete_collection(collection_name=collection_name)

client.create_collection(  # recreate 可避免重複建立
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)



# 3. 上傳向量與 payload
points = []

for i, doc in enumerate(documents):
    full_text = doc['question'] + ' ' + doc['text']
    vector = models.Document(text=full_text, model=model_handle)
    point = models.PointStruct(
        id=i,
        vector=vector,
        payload=doc
    )
    points.append(point)
    
    
client.upsert(collection_name=collection_name, points=points)

def search(query, limit=1):
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model=model_handle
        ),
        limit=limit,
        with_payload=True
    )
    return results

# 4. 查詢最相似的問句

res = search("I just discovered the course. Can I join now?", limit=1)
# print("最相似的文件 id:", res[0].id)
# print("內容:", res[0].payload)

print(res)
point = res.points[0]

print(f"最相似文件 ID: {point.id}")
print(f"相似度分數: {point.score:.4f}")
print(f"問題: {point.payload['question']}")
print(f"內容摘要: {point.payload['text'][:200]}...")


points=[ScoredPoint(id=0, version=0, score=0.85145426, payload={'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}, vector=None, shard_key=None, order_value=None)]
最相似文件 ID: 0
相似度分數: 0.8515
問題: Course - Can I still join the course after the start date?
內容摘要: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the las...


# Q5. Selecting the embedding model


In [9]:
from fastembed.embedding import TextEmbedding

models = TextEmbedding.list_supported_models()

# 找出最小維度的模型
min_model = min(models, key=lambda x: x['dim'])

print(f"最小維度模型: {min_model['model']} ({min_model['dim']} dimensions)")


2025-06-28 08:34:41.068 | WARNING  | fastembed.embedding:<module>:7 - DefaultEmbedding, FlagEmbedding, JinaEmbedding are deprecated.Use from fastembed import TextEmbedding instead.


最小維度模型: BAAI/bge-small-en (384 dimensions)


In [10]:
from fastembed.embedding import TextEmbedding

embedder = TextEmbedding(model_name="BAAI/bge-small-en")

# embed 回傳 generator，要先轉成 list
vectors = list(embedder.embed(["Hello world!"]))
vector = vectors[0]

print(f"Embedding vector dimension: {len(vector)}")  # 預期為 384


Embedding vector dimension: 384


# Q6. Indexing with qdrant (2 points)


In [11]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)
        
       

In [12]:
documents[:5]

[{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
  'section': 'General course-related questions',
  'question': 'How do I sign up?',
  'course': 'machine-learning-zoomcamp'},
 {'text': 'The course videos are pre-recorded, you can start watching the course right now.\nWe will also occasionally have office hours - live sessions where we will answer your questions. The office hours sessions are recorded too.\nYou can see the office hours as well as the pre-recorded course videos in the course playlist on YouTube.',
  'section': 'General course-related questions',
  'question': 'Is it going to be live? When?',
  'course': 'machi

In [20]:
collection_name = "question6"

# 2. 建立 Qdrant collection
client.delete_collection(collection_name=collection_name)

client.create_collection(  # recreate 可避免重複建立
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        
        size=384,
        distance=models.Distance.COSINE
    )
)

True

In [16]:
points = []
id = 0
from qdrant_client import QdrantClient, models
for course in documents_raw:
    for doc in course['documents']:
        text = doc['question'] + ' ' + doc['text']
        point = models.PointStruct(
            id=id,
            vector=models.Document(text=text, model="BAAI/bge-small-en"), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1
        

In [21]:
client.upsert(
    collection_name=collection_name,  # collection name
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [24]:

def search(query, limit=1):
    results = client.query_points(
        collection_name=collection_name,
        query=models.Document(
            text=query,
            model="BAAI/bge-small-en"
        ),
        limit=limit,
        with_payload=True
    )
    return results

# 4. 查詢最相似的問句

res = search("I just discovered the course. Can I join now?", limit=1)
print(res)

points=[ScoredPoint(id=449, version=0, score=0.8703172, payload={'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.', 'section': 'General course-related questions', 'course': 'machine-learning-zoomcamp'}, vector=None, shard_key=None, order_value=None)]
